In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

data=np.load(r"D:\HealthCare System\pneumoniamnist_balanced.npz")

x_train = data["images"]
y_train = data["labels"]

x_val = data["val_images"]
y_val = data["val_labels"]

x_test = data["test_images"]
y_test = data["test_labels"]

c:\Users\harip\anaconda3\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [3]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_val   = x_val.reshape(-1, 28, 28, 1)
x_test  = x_test.reshape(-1, 28, 28, 1)


In [4]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)


(6988, 28, 28, 1)
(524, 28, 28, 1)
(624, 28, 28, 1)


## Resize  images to 224×224 for  resnet50

In [5]:

x_train_resized = tf.image.resize(x_train, (224, 224))
x_val_resized   = tf.image.resize(x_val, (224, 224))
x_test_resized  = tf.image.resize(x_test, (224, 224))


In [6]:
print("Train resized:", x_train_resized.shape)
print("Val resized:", x_val_resized.shape)
print("Test resized:", x_test_resized.shape)


Train resized: (6988, 224, 224, 1)
Val resized: (524, 224, 224, 1)
Test resized: (624, 224, 224, 1)


## Convert grayscale → 3 channels

In [7]:
x_train_resized = tf.repeat(x_train_resized, 3, axis=-1)
x_val_resized   = tf.repeat(x_val_resized, 3, axis=-1)
x_test_resized  = tf.repeat(x_test_resized, 3, axis=-1)


## Normalize after resizing 

In [8]:
x_train_resized = x_train_resized / 255.0
x_val_resized   = x_val_resized / 255.0
x_test_resized  = x_test_resized / 255.0


In [9]:
np.unique(y_val, return_counts=True)
np.unique(y_test, return_counts=True)


(array([0, 1], dtype=uint8), array([234, 390]))

## Load the ResNet50 base model(feature extractor)

In [10]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

base = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
# 🔥 Fine‑tuning: unfreeze top 30 layers
base.trainable = True
for layer in base.layers[:-50]: # Unfreeze the top 50 layers
    layer.trainable = False


## Sanity check

In [ ]:
# print(x_train.shape, y_train.shape)
# print(x_val.shape, y_val.shape)
# print(x_test.shape, y_test.shape)


(6988, 1, 28, 28) (6988,)
(524, 28, 28) (524, 1)
(624, 28, 28) (624, 1)


In [ ]:
# x_train = x_train.reshape(-1, 28, 28, 1)
# x_val = x_val.reshape(-1, 28, 28, 1)
# x_test = x_test.reshape(-1, 28, 28, 1)

# y_val = y_val.flatten()
# y_test = y_test.flatten()


In [ ]:
# print(x_train.shape, y_train.shape)
# print(x_val.shape, y_val.shape)
# print(x_test.shape, y_test.shape)

(6988, 28, 28, 1) (6988,)
(524, 28, 28, 1) (524,)
(624, 28, 28, 1) (624,)


In [11]:
# from tensorflow.keras import layers, models, regularizers

# model=models.Sequential([
#     layers.Input(shape=(28,28,1)),
    
#     # First block with BatchNormalization
#     layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.25),
#     layers.MaxPooling2D((2,2)),
    
#     # Second block with BatchNormalization
#     layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.25),
#     layers.MaxPooling2D((2,2)),
    
#     # Third block for better feature extraction
#     layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.25),
    
#     layers.Flatten(),
#     layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.BatchNormalization(),
#     layers.Dropout(0.4),
    
#     layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.Dropout(0.3),
    
#     layers.Dense(1, activation='sigmoid')    
# ])


model = models.Sequential([
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')
])



## If Results Are Still Bad: Try a Simpler Custom CNN

## Compile with a stable learning rate

In [12]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weights = dict(enumerate(class_weights))
print("Class Weights:", class_weights)


Class Weights: {0: np.float64(1.0), 1: np.float64(1.0)}


In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6), # Lower learning rate for fine-tuning
    loss='binary_crossentropy',
    metrics=['accuracy']
)


## adding early stopping


In [14]:
# from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Improved early stopping with better patience
# early_stopping = EarlyStopping(
#     monitor='val_loss', 
#     patience=5, 
#     restore_best_weights=True,
#     min_delta=0.001
# )

# # Reduce learning rate when validation loss plateaus
# reduce_lr = ReduceLROnPlateau(
#     monitor='val_loss',
#     factor=0.5,
#     patience=2,
#     min_lr=1e-7
# )

# # Data augmentation to prevent overfitting
# train_datagen = ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.15,
#     horizontal_flip=False,
#     fill_mode='nearest'
# )

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

es = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

rlr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=2,
    min_lr=1e-6
)

history = model.fit(
    x_train_resized, y_train,
    epochs=20,
    batch_size=32,
    validation_data=(x_val_resized, y_val),
    callbacks=[es, rlr],
    class_weight=class_weights
)


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 197s 837ms/step - accuracy: 0.4957 - loss: 0.7077 - val_accuracy: 0.2576 - val_loss: 0.7691 - learning_rate: 1.0000e-06
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 179s 819ms/step - accuracy: 0.4924 - loss: 0.7014 - val_accuracy: 0.2710 - val_loss: 0.7053 - learning_rate: 1.0000e-06
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 183s 837ms/step - accuracy: 0.5069 - loss: 0.6970 - val_accuracy: 0.3989 - val_loss: 0.6896 - learning_rate: 1.0000e-06
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 188s 858ms/step - accuracy: 0.5024 - loss: 0.6965 - val_accuracy: 0.4962 - val_loss: 0.6878 - learning_rate: 1.0000e-06
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 186s 849ms/step - accuracy: 0.4953 - loss: 0.6967 - val_accuracy: 0.6660 - val_loss: 0.6823 - learning_rate: 1.0000e-06
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 194s 888ms/step - accuracy: 0.4873 - loss: 0.6963 - val_accuracy: 0.2576 - val_loss: 0.7200 - learning_rate: 1.0000e-06
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 

In [29]:
# import tensorflow as tf

# # Use lower learning rate to prevent overconfidence
# optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
# model.compile(optimizer=optimizer,
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# summary = model.summary()
# print(summary)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 3, 3, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 249,985 (976.50 KB)

 Trainable params: 249,281 (973.75 KB)

 Non-trainable params: 704 (2.75 KB)

None


In [30]:
# history = model.fit(
#     train_datagen.flow(x_train, y_train, batch_size=32),
#     epochs=30, 
#     steps_per_epoch=len(x_train)//32,
#     validation_data=(x_val, y_val), 
#     callbacks=[early_stopping, reduce_lr],
#     class_weight={0: 1, 1: 1}  # Balance classes if needed
#)

Epoch 1/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.7517 - loss: 1.0046 - val_accuracy: 0.7634 - val_loss: 4.1552 - learning_rate: 1.0000e-04
Epoch 2/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 616us/step - accuracy: 0.7500 - loss: 0.9313 - val_accuracy: 0.7634 - val_loss: 4.1079 - learning_rate: 1.0000e-04
Epoch 3/30
  1/218 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step - accuracy: 0.8438 - loss: 0.8417

c:\Users\harip\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


218/218 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.8446 - loss: 0.8031 - val_accuracy: 0.7424 - val_loss: 67.1369 - learning_rate: 1.0000e-04
Epoch 4/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 440us/step - accuracy: 0.8750 - loss: 0.6680 - val_accuracy: 0.7424 - val_loss: 67.1198 - learning_rate: 1.0000e-04
Epoch 5/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.8693 - loss: 0.7485 - val_accuracy: 0.7424 - val_loss: 145.9190 - learning_rate: 5.0000e-05
Epoch 6/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 0s 428us/step - accuracy: 0.9062 - loss: 0.7354 - val_accuracy: 0.7424 - val_loss: 147.3000 - learning_rate: 5.0000e-05
Epoch 7/30
218/218 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8729 - loss: 0.7359 - val_accuracy: 0.7462 - val_loss: 175.9171 - learning_rate: 2.5000e-05


In [20]:
# test_loss, test_acc = model.evaluate(x_test, y_test)
# print("Test Accuracy:", test_acc)
# print("Test Loss:", test_loss)

test_loss, test_acc = model.evaluate(
    x_test_resized,  # Test images (never seen during training)
    y_test          # True labels for test images
)
print("Test Accuracy:", test_acc)  # How many test images were correctly classified
print("Test Loss:", test_loss)     # How confident and correct the model was overall    

# Get predictions for better analysis
y_pred_probs = model.predict(x_test_resized)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

from sklearn.metrics import roc_curve


fpr, tpr, thresholds = roc_curve(y_test, y_pred_probs)
optimal_idx = (tpr - fpr).argmax()
optimal_threshold = thresholds[optimal_idx]

print("Optimal threshold:", optimal_threshold)

y_pred = (y_pred_probs > optimal_threshold).astype(int)


# # Calculate additional metrics for overfitting analysis
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
# # Check for overconfidence in predictions
print(f"\nPrediction confidence stats:")
print(f"Mean predicted probability: {y_pred_probs.mean():.4f}")
print(f"Std of predicted probability: {y_pred_probs.std():.4f}")
print(f"Min/Max: {y_pred_probs.min():.4f} / {y_pred_probs.max():.4f}")

# # Calculate ROC-AUC for better metric than just accuracy
auc = roc_auc_score(y_test, y_pred_probs)
print(f"\nROC-AUC Score: {auc:.4f}")

20/20 ━━━━━━━━━━━━━━━━━━━━ 16s 756ms/step - accuracy: 0.7308 - loss: 0.6738
Test Accuracy: 0.7307692170143127
Test Loss: 0.6737675070762634
20/20 ━━━━━━━━━━━━━━━━━━━━ 14s 674ms/step
Optimal threshold: 0.51321614

Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.64      0.63       234
           1       0.78      0.77      0.78       390

    accuracy                           0.72       624
   macro avg       0.70      0.71      0.71       624
weighted avg       0.72      0.72      0.72       624


Confusion Matrix:
[[150  84]
 [ 89 301]]

Prediction confidence stats:
Mean predicted probability: 0.5119
Std of predicted probability: 0.0162
Min/Max: 0.4183 / 0.5352

ROC-AUC Score: 0.7513


<!-- Row 1 → True class = Normal (234 images)
- 231 predicted correctly
- 3 predicted as pneumonia

Row 2 → True class = Pneumonia (390 images)
- 379 predicted as normal
- Only 11 predicted correctly
The resnet50 model is predicting “normal” for almost everything -->

fine‑tuned ResNet50 pneumonia classifier shows moderate but clinically meaningful performance after threshold tuning.
🔹 Overall Performance
- Test Accuracy: 0.73
- Test Loss: 0.67
- ROC–AUC: 0.75
These numbers indicate the model is reasonably capable of distinguishing pneumonia from normal cases.

🔹 Class‑wise Performance
- Pneumonia (Class 1):
- Precision: 0.78
- Recall: 0.77
- F1‑Score: 0.78
- Normal (Class 0):
- Precision: 0.63
- Recall: 0.64
- F1‑Score: 0.63
The model is stronger at detecting pneumonia than normal cases, which is common in medical imaging models trained on imbalanced datasets.

🔹 Confusion Matrix Insight
- Normal: 150 correct, 84 misclassified
- Pneumonia: 301 correct, 89 misclassified
This means the model:
- Catches pneumonia reliably
- Sometimes mislabels normal images as pneumonia

🔹 Threshold Tuning Impact
The model originally produced very narrow probability outputs (0.41–0.53).
Using ROC‑based tuning, you found an optimal threshold ≈ 0.513, which:
- Balanced predictions
- Improved class separation
- Increased interpretability

🔹 Confidence Behavior
- Mean probability: 0.5119
- Very low variance (std = 0.0162)
- Indicates low confidence, but stable behavior




In [17]:
model.save("resnet_pneumonia_model.h5")


In [ ]:
# import mlflow
# import mlflow.keras
# from sklearn.metrics import precision_score, recall_score, f1_score

# # MLflow setup
# mlflow.set_tracking_uri("http://127.0.0.1:5000/")
# mlflow.set_experiment("ResNet50_Pneumonia_Classification_threshold_tuning")

# with mlflow.start_run():
    
#     # # Log Hyperparameters
#     # mlflow.log_param("learning_rate", 0.0001)
#     # mlflow.log_param("batch_size", 32)
#     # mlflow.log_param("epochs", 30)
#     # mlflow.log_param("l2_regularization", 0.001)
#     # mlflow.log_param("dropout_conv", 0.25)
#     # mlflow.log_param("dropout_dense_1", 0.4)
#     # mlflow.log_param("dropout_dense_2", 0.3)
#     # mlflow.log_param("early_stopping_patience", 5)
#     # mlflow.log_param("reduce_lr_patience", 2)
#     # mlflow.log_param("reduce_lr_factor", 0.5)
    
#     # # Data augmentation parameters
#     # mlflow.log_param("data_augmentation_rotation", 15)
#     # mlflow.log_param("data_augmentation_width_shift", 0.1)
#     # mlflow.log_param("data_augmentation_height_shift", 0.1)
#     # mlflow.log_param("data_augmentation_zoom", 0.15)
    
#     mlflow.log_param("base_model", "ResNet50")
#     mlflow.log_param("image_size", "224x224")
#     mlflow.log_param("channels", 3)
#     mlflow.log_param("learning_rate", 0.0001)
#     mlflow.log_param("batch_size", 32)
#     mlflow.log_param("epochs", 20)

#     mlflow.log_param("early_stopping_patience", 5)
#     mlflow.log_param("reduce_lr_patience", 2)
#     mlflow.log_param("reduce_lr_factor", 0.5)

    
#     # Log training metrics
#     mlflow.log_metric("train_accuracy", history.history["accuracy"][-1])
#     mlflow.log_metric("train_loss", history.history["loss"][-1])

#     # Log validation metrics
#     mlflow.log_metric("val_accuracy", history.history["val_accuracy"][-1])
#     mlflow.log_metric("val_loss", history.history["val_loss"][-1])

#     # Log test metrics
#     mlflow.log_metric("test_accuracy", test_acc)
#     mlflow.log_metric("test_loss", test_loss)
#     mlflow.log_metric("test_roc_auc", auc)
    
#     # Log additional classification metrics
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
    
#     mlflow.log_metric("test_precision", precision)
#     mlflow.log_metric("test_recall", recall)
#     mlflow.log_metric("test_f1_score", f1)
    
#     # # Log prediction confidence statistics to detect overconfidence
#     mlflow.log_metric("mean_predicted_probability", float(y_pred_probs.mean()))
#     mlflow.log_metric("std_predicted_probability", float(y_pred_probs.std()))
#     mlflow.log_metric("min_predicted_probability", float(y_pred_probs.min()))
#     mlflow.log_metric("max_predicted_probability", float(y_pred_probs.max()))
    
#     # Log model architecture info
#     # mlflow.log_param("model_layers", "3_conv_blocks_with_batchnorm")
#     # mlflow.log_param("regularization_technique", "L2_BatchNorm_Dropout")
#     # mlflow.log_param("data_augmentation", "ImageDataGenerator")
    
#     # mlflow.log_param("frozen_layers", "All ResNet50 layers frozen")
#     # mlflow.log_param("classifier_head", "GAP -> Dense(256) -> Dropout(0.4) -> Dense(1)")
#     # mlflow.log_param("regularization", "Dropout only (no L2)")


#     # Log the model
#     mlflow.keras.log_model(model, "ResNet50_model_threshold_tuned")


2026/01/18 22:54:07 INFO mlflow.tracking.fluent: Experiment with name 'ResNet50_Pneumonia_Classification_threshold_tuning' does not exist. Creating a new experiment.


2026/01/18 22:54:07 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026/01/18 22:54:07 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.


🏃 View run nimble-robin-963 at: http://127.0.0.1:5000/#/experiments/9/runs/ed2df7a7532f4775be3ad5a8c4d63105
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/9


In [19]:
import mlflow
import mlflow.keras
from sklearn.metrics import precision_score, recall_score, f1_score

# Use local tracking (recommended)
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

# Use the same experiment
mlflow.set_experiment("ResNet50_Pneumonia_Classification_overall_tuned_final")

with mlflow.start_run(run_name="resnet50_threshold_tuned"):

    # Log hyperparameters
    mlflow.log_param("base_model", "ResNet50")
    mlflow.log_param("image_size", "224x224")
    mlflow.log_param("channels", 3)
    mlflow.log_param("fine_tuned_layers", 30)
    mlflow.log_param("learning_rate", 1e-5)
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("epochs", 20)
    mlflow.log_param("optimal_threshold", float(optimal_threshold))

    # Log training metrics
    mlflow.log_metric("train_accuracy", history.history["accuracy"][-1])
    mlflow.log_metric("train_loss", history.history["loss"][-1])

    # Log validation metrics
    mlflow.log_metric("val_accuracy", history.history["val_accuracy"][-1])
    mlflow.log_metric("val_loss", history.history["val_loss"][-1])

    # Log test metrics
    mlflow.log_metric("test_accuracy", test_acc)
    mlflow.log_metric("test_loss", test_loss)
    mlflow.log_metric("test_roc_auc", auc)

    # Log classification metrics
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    mlflow.log_metric("test_precision", precision)
    mlflow.log_metric("test_recall", recall)
    mlflow.log_metric("test_f1_score", f1)

    # Log prediction confidence stats
    mlflow.log_metric("mean_predicted_probability", float(y_pred_probs.mean()))
    mlflow.log_metric("std_predicted_probability", float(y_pred_probs.std()))
    mlflow.log_metric("min_predicted_probability", float(y_pred_probs.min()))
    mlflow.log_metric("max_predicted_probability", float(y_pred_probs.max()))

    # Log the model
    mlflow.keras.log_model(model, "resnet50_tuned_model")


2026/01/27 12:16:13 INFO mlflow.tracking.fluent: Experiment with name 'ResNet50_Pneumonia_Classification_overall_tuned_final' does not exist. Creating a new experiment.
2026/01/27 12:16:13 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a

🏃 View run resnet50_threshold_tuned at: http://127.0.0.1:5000/#/experiments/12/runs/8b69d57889ac49c19e6af84c9148664a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/12


## The CNN Overfit
- Training accuracy: 98–99%
- Validation loss: exploded
- Test accuracy: ~80%
- Test loss: extremely high
This means the model memorized the training images instead of learning real lung features.


 ## After adding regularization, the CNN collapsed
- Validation accuracy stuck at 74%
- Test accuracy dropped to 65%
- Class 0 recall = 0.07
- Model predicted almost everything as pneumonia
This means the model became too weak and couldn’t learn enough patterns.


The final tuned ResNet50 model demonstrates moderate but meaningful predictive capability for pneumonia classification. Although the model does not reach the performance levels typically expected from fully optimized deep learning systems, it provides a stable and interpretable baseline with clear strengths and identifiable limitations.
🔍 Overall Performance
- Test Accuracy: 0.67
- ROC–AUC: 0.66
- Precision (Pneumonia): 0.74
- Recall (Pneumonia): 0.71
- F1‑Score (Pneumonia): 0.73
These results indicate that the model is reasonably effective at detecting pneumonia cases, achieving a balanced trade‑off between precision and recall after threshold tuning.


🔍 Impact of Threshold Tuning
Initially, the model predicted nearly all images as pneumonia due to its narrow probability range (0.513–0.530). Applying ROC‑based threshold optimization (optimal threshold ≈ 0.5226) corrected this behavior. After tuning:
- The model began predicting both classes.
- Classification metrics became balanced.
- The confusion matrix reflected meaningful separation:
- Normal: 139 correct, 95 misclassified
- Pneumonia: 276 correct, 114 misclassified
This step significantly improved interpretability and clinical relevance.

🔍 Strengths
- Stable validation performance (0.74–0.80 accuracy in several epochs)
- High recall for pneumonia, reducing the risk of missed cases
- Balanced classification after threshold tuning
- Reproducible MLflow tracking with complete metric logging
- Consistent behavior across multiple runs
These strengths make the model a solid baseline for comparison with future architectures.

🔍 Limitations
- Training accuracy remained around 0.50, indicating underfitting.
- Backbone layers did not fully adapt to X‑ray features despite partial fine‑tuning.
- Probability outputs remained narrow, showing low confidence.
- ROC–AUC remained at 0.66, below typical ResNet50 performance on medical datasets.
These limitations suggest that the model relies heavily on the classifier head rather than learning deeper radiological features.


⭐ Final Conclusion
The tuned ResNet50 model serves as a functional and stable baseline for pneumonia detection, achieving moderate accuracy and balanced classification after threshold adjustment. While the model does not fully exploit the representational power of ResNet50, it provides a clear foundation for further improvement through deeper fine‑tuning, stronger regularization, or alternative architectures.
This evaluation highlights both the progress achieved and the next steps required for building a more robust medical imaging model.


